<hr>
Este ejemplo toma parte de implementaciones de las siguientes fuentes:
- https://pythonprogramming.net/convolutional-neural-network-kats-vs-dogs-machine-learning-tutorial/ y adaptado para el semillero de DeepLearningAplicado
- https://www.kaggle.com/stevenhurwitt/cats-vs-dogs-using-a-keras-convnet

# Full run through of raw images to classification with Convolutional Neural Network #

In this tutorial, we're going to be running through taking raw images that have been labeled for us already, and then feeding them through a convolutional neural network for classification. 

The images are either of dog(s) or cat(s). 

Once you have downloaded and extracted the data from https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data, you're ready to begin.

In [ ]:
import numpy as np         # dealing with arrays
import os                  # dealing with directories
from random import shuffle # mixing up or currently ordered data that might lead our network astray in training.
from tqdm import tqdm      # a nice pretty percentage bar for tasks. Thanks to viewer Daniel Bühler for this suggestion

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.layers import Dense, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
import matplotlib.pylab as plt

#TRAIN_DIR = 'dataset/dogscats/sample/train'
TRAIN_DIR = 'dataset/dogscats/train'
TEST_DIR = 'dataset/dogscats/sample/valid'
IMG_SIZE = 64
NUM_CLASSES = 2

Now, our first order of business is to convert the images and labels to array information that we can pass through our network. To do this, we'll need a helper function to convert the image name to an array. 

Our images are labeled like "cat.1" or "dog.3" and so on, so we can just split out the dog/cat, and then convert to an array like so:

In [ ]:
def label_img(img):
    word_label = img.split('.')[-3]
    # conversion to one-hot array [cat,dog]
    #                            [much cat, no dog]
    if word_label == 'cat': return [1,0]
    #                             [no cat, very doggo]
    elif word_label == 'dog': return [0,1]

Now, we can build another function to fully process the training images and their labels into arrays:

In [ ]:
from skimage.io import imread
from skimage.transform import resize

def load_dataset(folder_path):
    dir_list = os.listdir(folder_path)

    training_data = np.zeros([len(dir_list), IMG_SIZE, IMG_SIZE, 3])
    training_classes = np.zeros([len(dir_list), 2])
    
    i = 0
    for img in tqdm(dir_list):
        label = label_img(img)
        path = os.path.join(folder_path,img)
        
        img = imread(path, as_grey=True)
        img_res = resize(img, (IMG_SIZE, IMG_SIZE)).reshape(IMG_SIZE, IMG_SIZE, 1)
           
        training_data[i] = img_res
        training_classes[i] = label
        
        i += 1
    
    return (training_data, training_classes)

## Now, we can run the training:

In [ ]:
(train_data, train_classes) = load_dataset(TRAIN_DIR)
(test_data, test_classes) = load_dataset(TEST_DIR)

In [ ]:
print(train_len)
print(shuffle_index)

In [ ]:
train_len = len(train_data)
shuffle_index = np.random.permutation(train_len)

train_x = train_data[shuffle_index[0:1000]]
test_x = train_data[shuffle_index[1000:1500]]

train_y = train_classes[shuffle_index[0:1000]]
test_y = train_classes[shuffle_index[1000:1500]]

input_shape = (IMG_SIZE, IMG_SIZE, 3)

In [ ]:
import matplotlib.pyplot as plt

rand_index = np.random.randint(500)

plt.imshow(test_x[rand_index], cmap="gray")
plt.show()

print(test_y[rand_index])

In [ ]:
train_x = train_x.astype('float32') / 255.
test_x = test_x.astype('float32') / 255.

# Convolutional Neural Network: Keras

In [ ]:
#Model Architecture
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(1, 1)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(1, 1)))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(NUM_CLASSES, activation='softmax'))

#Model Compilation
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
model = Sequential()

#Convolutional block 1
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(5, 5), strides=(1, 1)))

#Convolutional block 2
model.add(Conv2D(64, kernel_size=(5, 5), strides=(1, 1),activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(5, 5), strides=(1, 1)))

#Convolutional block 3
model.add(Conv2D(128, kernel_size=(5, 5), strides=(1, 1), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(5, 5), strides=(1, 1)))

#Convolutional block 4
model.add(Conv2D(64, kernel_size=(5, 5), strides=(1, 1), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(5, 5), strides=(1, 1)))

#Convolutional block 3
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(5, 5), strides=(1, 1)))

#FCN + Dropout
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))

#Last layer Softmax
model.add(Dense(NUM_CLASSES, activation='softmax'))

#Model Compilation
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils

optimizer = keras.optimizers.Adam(lr=0.0001)
objective = 'binary_crossentropy'

input_shape = [None, IMG_SIZE, IMG_SIZE, 3]

def catdog():
    
    model = Sequential()

    model.add(Conv2D(32, 3, padding='same', input_shape=input_shape, activation='relu'))
    model.add(Conv2D(32, 3, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))
    #print("First layer...")
    model.add(Conv2D(64, 3, padding='same', activation='relu'))
    model.add(Conv2D(64, 3, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))
    #print("Second layer...")
    model.add(Conv2D(128, 3, padding='same', activation='relu'))
    model.add(Conv2D(128, 3, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))
    #print("Third layer...")
    model.add(Conv2D(256,  kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu'))
    model.add(Conv2D(256,  kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu'))
    model.add(Conv2D(256,  kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))

    #model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    #model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    #model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    #print("Flattening, etc...")
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(512, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    
    print("Compiling model...")
    model.compile(loss=objective, optimizer=optimizer, metrics=['accuracy'])
    print("Model compiled")
    
    return model

In [ ]:
model = catdog()

Now we fit for 3 epochs:

In [ ]:
batch_size = 32
num_epochs = 10

model.fit(train_x, train_y,
          batch_size=batch_size,
          epochs=num_epochs,
          verbose=1,
          validation_data=(test_x, test_y))

Hmm... it doesn't look like we've gotten anywhere at all. 